In [1]:
import os
import time
import datetime
from pathlib import Path
import numpy as np
import pandas as pd
import pyabf
import utils

%load_ext autoreload
%autoreload 2

In [2]:
human_meta = pd.read_csv('./data/raw/Metadata/Human19_Metadatav3.csv')
human_meta.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
human_meta.shape

(372, 16)

In [4]:
human_meta.head()

,ABF File,Data Type,Exp. Date,Cell #,Cell Layer,Stim Type,Threshold,DC,ZD,Hold,Gain,Offset,Response Channel,Command Channel,RMP,stimulus_type
0,2019_11_26_0010,Human,Nov_26_2019,C1,L5,Gap Free,n.a,n.a,ZD,n.a,1.0,0.0,Vm_Sec(mV),NaN,-71.5,NaN
1,2019_11_26_0011,Human,Nov_26_2019,C1,L5,Gap Free,n.a,n.a,ZD,n.a,1.0,0.0,Vm_Sec(mV),NaN,-71.5,NaN
2,2019_11_26_0018,Human,Nov_26_2019,C2,L5,Gap Free,n.a,n.a,ZD,n.a,1.0,0.0,Vm_Sec(mV),NaN,-72.9,NaN
3,2019_11_26_0020,Human,Nov_26_2019,C2,L5,Gap Free,n.a,n.a,ZD,n.a,1.0,0.0,Vm_Sec(mV),NaN,-72.9,NaN
4,2019_11_26_0021,Human,Nov_26_2019,C2,L5,Gap Free,n.a,n.a,ZD,n.a,1.0,0.0,Vm_Sec(mV),NaN,-72.9,NaN


In [5]:
intrinsic_meta = human_meta[human_meta['Stim Type'] == 'Intrinsic Properties']

In [6]:
intrinsic_meta.shape

(71, 16)

## Identify files to be converted
- ABF files from human2019
- need metadata to determine which files are testing intrinsic properties
- extract ABF version info, time of creation and comments from the ABF file

In [7]:
human2019_abf_dir = Path("./data/raw/recordings/Human tissue/Human tissue-White noise_2019/")
human2019_file_info = utils.summary_df(human2019_abf_dir)

In [8]:
%%time
def parse_timestamp(abf_path):
    abf = pyabf.ABF(abf_path)
    return datetime.datetime.strptime(abf.abfDateTimeString, "%Y-%m-%dT%H:%M:%S.%f")

def extract_abf_version(abf_path):
    abf = pyabf.ABF(abf_path)
    return abf.abfVersionString

def extract_abf_comments(abf_path):
    abf = pyabf.ABF(abf_path)
    return abf.tagComments

human2019_file_info['timestamp'] = human2019_file_info.path.apply(parse_timestamp)
human2019_file_info['abf_version'] = human2019_file_info.path.apply(extract_abf_version)
human2019_file_info['abf_comments'] = human2019_file_info.path.apply(extract_abf_comments)

# add this column to merge with provided metadata
human2019_file_info['file_id'] = human2019_file_info.file_name.apply(lambda x: Path(x).stem) 

CPU times: user 19.5 s, sys: 8.9 s, total: 28.4 s
Wall time: 31.1 s


In [9]:
human2019_file_info.shape

(503, 6)

In [10]:
human2019_file_info.file_name.nunique()

488

In [11]:
human2019_file_info.head()

,file_name,path,timestamp,abf_version,abf_comments,file_id
0,19219023.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-02-19 15:18:15.656,1.8.3.0,"[C3,L5, RMP -65.6 mv, gain 40; DC 200]",19219023
1,19219021.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-02-19 15:14:56.781,1.8.3.0,"[C3,L5, RMP -65.6 mv,]",19219021
2,19219005.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-02-19 12:13:46.656,1.8.3.0,"[C1,L5, RMP -62.6 mv,]",19219005
3,19219004.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-02-19 12:13:30.968,1.8.3.0,"[C1,L5, RMP -62.6 mv,]",19219004
4,19219016.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-02-19 12:45:01.828,1.8.3.0,"[C1,L5, RMP -61.6 mv, Gain 40; DC 150 pA]",19219016


In [12]:
human2019_file_info.abf_version.value_counts()

2.6.0.0    276
1.8.3.0    227
Name: abf_version, dtype: int64

## Merge metadata with extracted information about ABF files

In [13]:
merged_meta = intrinsic_meta.merge(human2019_file_info, left_on='ABF File', right_on='file_id' , how='left')

In [14]:
# we are left with 36 files to convert
merged_meta.abf_version.value_counts()

1.8.3.0    37
2.6.0.0    36
Name: abf_version, dtype: int64

In [15]:
merged_meta['ABF File'].nunique()

71

In [16]:
merged_meta.columns

Index(['ABF File', 'Data Type', 'Exp. Date', 'Cell #', 'Cell Layer',
       'Stim Type', 'Threshold', 'DC', 'ZD', 'Hold', 'Gain', 'Offset',
       'Response Channel', 'Command Channel', 'RMP', 'stimulus_type',
       'file_name', 'path', 'timestamp', 'abf_version', 'abf_comments',
       'file_id'],
      dtype='object')

## Convert

In [17]:
output_dir = Path("data/processed/recordings/human2019/")
output_dir.mkdir(exist_ok=True, parents=True)

In [18]:
from x_to_nwb import ABF2Converter
from x_to_nwb import ABF1Converter_meta2

def generate_subject_meta(row):
    meta = {
    'Subject': {
        'age': None,
        'description': f'{row.abf_comments}',
        'species': 'Homo sapiens',
        'subject_id': f'Donor_{row.file_id}'}
    }
    return meta


def convert_abfv1_row(row):
    abf = pyabf.ABF(row.path)
    file_stem = row.file_id
    output_path = output_dir / f'{file_stem}.nwb'
    meta = generate_subject_meta(row)

    nwb = ABF1Converter_meta2.ABF1Converter(row.path,
                                      str(output_path), 
                                      acquisitionChannelName= row.resp_chan, #'Iclamp',#row.stim_chan,
                                      stimulusChannelName=row.stim_chan, #'Cmd 1', #row.resp_chan,
                                      responseOffset=float(row.Offset) * (-1),# * (1e-3),
                                      metadata=meta,
                                      responseGain=float(row.Gain) *0.01)
    
    nwb.convert()
    print(f"NWB Conversion complete for {output_path}")


def convert_abfv2_row(row):
    abf = pyabf.ABF(row.path)
    file_stem = row.file_id
    output_path = output_dir / f'{file_stem}.nwb'
    meta = generate_subject_meta(row)
    print(f'Subject meta: {meta}')
    print(f'Stimulus chan: {row.stim_chan}')
    print(f'Response chan: {row.resp_chan}')

    nwb = ABF2Converter.ABF2Converter(row.path,
                                      str(output_path), 
                                      includeChannelList=[row.resp_chan],
                                      stimulus_name=row.stim_chan,
                                      metadata=meta)

    print(f"NWB Conversion complete for {output_path}")

In [19]:
merged_meta.shape

(73, 22)

In [20]:
merged_meta[merged_meta.duplicated(subset=['file_id'], keep=False)]

,ABF File,Data Type,Exp. Date,Cell #,Cell Layer,Stim Type,Threshold,DC,ZD,Hold,...,Response Channel,Command Channel,RMP,stimulus_type,file_name,path,timestamp,abf_version,abf_comments,file_id
18,19129000,Human,Jan_29_2019,C1,L5,Intrinsic Properties,n.a,n.a,n.a,n.a,...,Iclamp(mV),Current_in(pA),-62.9,NaN,19129000.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-01-29 12:16:18.000,1.8.3.0,"[C1, L5, RMP -62.9mv,]",19129000
19,19129000,Human,Jan_29_2019,C1,L5,Intrinsic Properties,n.a,n.a,n.a,n.a,...,Iclamp(mV),Current_in(pA),-62.9,NaN,19129000.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-01-29 12:16:18.000,1.8.3.0,"[C1, L5, RMP -62.9mv,]",19129000
29,19319020,Human,March_19_2019,C4,L5,Intrinsic Properties,n.a,n.a,n.a,n.a,...,Iclamp(mV),Current_in(pA),-63.1,NaN,19319020.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-03-19 16:17:49.781,1.8.3.0,"[C4, L5, RMP -63.1mv,, C4, L5, RMP -63.1mv,]",19319020
30,19319020,Human,March_19_2019,C4,L5,Intrinsic Properties,n.a,n.a,n.a,n.a,...,Iclamp(mV),Current_in(pA),-63.1,NaN,19319020.abf,data/raw/recordings/Human tissue/Human tissue-...,2019-03-19 16:17:49.781,1.8.3.0,"[C4, L5, RMP -63.1mv,, C4, L5, RMP -63.1mv,]",19319020


In [21]:
abf1 = merged_meta[merged_meta.abf_version.str.startswith('1')]
abf2 = merged_meta[merged_meta.abf_version.str.startswith('2')]

In [22]:
abf1['stim_chan'] = abf1.path.apply(lambda x: pyabf.ABF(x).dacNames[1])
abf1['resp_chan'] = abf1.path.apply(lambda x: pyabf.ABF(x).adcNames[2])

abf2['stim_chan'] = abf2.path.apply(lambda x: pyabf.ABF(x).dacNames[1])
abf2['resp_chan'] = abf2.path.apply(lambda x: pyabf.ABF(x).adcNames[0])

<ipython-input-22-2ec584e15cec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abf1['stim_chan'] = abf1.path.apply(lambda x: pyabf.ABF(x).dacNames[1])
<ipython-input-22-2ec584e15cec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abf1['resp_chan'] = abf1.path.apply(lambda x: pyabf.ABF(x).adcNames[2])
<ipython-input-22-2ec584e15cec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [23]:
to_convert = pd.concat([abf1, abf2])
to_convert = to_convert[~to_convert.duplicated(subset=['file_id'], keep=False)]

In [24]:
to_convert.shape

(69, 24)

In [25]:
successes = []
errors = []

for i, row in to_convert.iterrows():
    if row.abf_version == '2.6.0.0':
        try:
            convert_abfv2_row(row)
            successes.append((row.file_id, row.path))
        except (ValueError, IndexError, TypeError, KeyError) as e:
            print('**'*50)
            print(f'Error: {e} \n {row.path} \n')
            errors.append((e, row.file_id, row.path))
    
    elif row.abf_version == '1.8.3.0':
        try:
            convert_abfv1_row(row)
            successes.append((row.file_id, row.path))
        except (ValueError, IndexError, TypeError, KeyError) as e:
            print('**'*50)
            print(f'Error: {e} \n {row.path} \n')
            errors.append((e, row.file_id, row.path))        

data/raw/recordings/Human tissue/Human tissue-White noise_2019/Feb_28_2019/C3_L5/Intrinsic Properties_C3_L5/19228030.abf


/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


Successfully converted to data/processed/recordings/human2019/19228030.nwb.
NWB Conversion complete for data/processed/recordings/human2019/19228030.nwb
data/raw/recordings/Human tissue/Human tissue-White noise_2019/Feb_28_2019/C4_L5/Intrinsic Properties_C4_L5/19228044.abf
Successfully converted to data/processed/recordings/human2019/19228044.nwb.
NWB Conversion complete for data/processed/recordings/human2019/19228044.nwb
data/raw/recordings/Human tissue/Human tissue-White noise_2019/Feb_28_2019/C5_L5/Intrinsic Properties_C5_L5/19228058.abf
Successfully converted to data/processed/recordings/human2019/19228058.nwb.
NWB Conversion complete for data/processed/recordings/human2019/19228058.nwb
data/raw/recordings/Human tissue/Human tissue-White noise_2019/Feb_28_2019/C6_L5/Intrinsic Properties_C6_L5/19228068.abf
Successfully converted to data/processed/recordings/human2019/19228068.nwb.
NWB Conversion complete for data/processed/recordings/human2019/19228068.nwb
data/raw/recordings/Human

Successfully converted to data/processed/recordings/human2019/19328034.nwb.
NWB Conversion complete for data/processed/recordings/human2019/19328034.nwb
data/raw/recordings/Human tissue/Human tissue-White noise_2019/March_28_2019/C5_L5/Intrinsic Properties_C5_L5/19328039.abf
Successfully converted to data/processed/recordings/human2019/19328039.nwb.
NWB Conversion complete for data/processed/recordings/human2019/19328039.nwb
data/raw/recordings/Human tissue/Human tissue-White noise_2019/March_28_2019/C6_L5/Intrinsic Properties_C6_L5/19328046.abf


/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:188: UserWarning: Could not find the JSON file data/raw/recordings/Human tissue/Human tissue-White noise_2019/Nov_04_2019/C2_L3/ZD_C2_L3C/Intrinsic Properties_C2_L3C/2019_11_04_0098.json with settings.
  warnings.warn(f"Could not find the JSON file {settings} with settings.")


...

NWB Conversion complete for data/processed/recordings/human2019/19o10062.nwb
Subject meta: {'Subject': {'age': None, 'description': "['C6, L2, APV,CNQX, PTX,']", 'species': 'Homo sapiens', 'subject_id': 'Donor_19o10063'}}
Stimulus chan: Cmd 1
Response chan: Vm_sec


/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:188: UserWarning: Could not find the JSON file data/raw/recordings/Human tissue/Human tissue-White noise_2019/October_10_2019_Epileptogenic zone/C6_L2/Intrinsic Properties_C6_L2/19o10065.json with settings.
  warnings.warn(f"Could not find the JSON file {settings} with settings.")


NWB Conversion complete for data/processed/recordings/human2019/19o10063.nwb
Subject meta: {'Subject': {'age': None, 'description': "['C6, L2, APV,CNQX, PTX,', 'C6, L2, APV,CNQX, PTX,']", 'species': 'Homo sapiens', 'subject_id': 'Donor_19o10065'}}
Stimulus chan: I_clampsec
Response chan: Vm_sec


/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:188: UserWarning: Could not find the JSON file data/raw/recordings/Human tissue/Human tissue-White noise_2019/Sep_03-2019/C1_L2/Intrinsic Properties_C1_L2/2019_09_03_0017.json with settings.
  warnings.warn(f"Could not find the JSON file {settings} with settings.")
/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:519: UserWarning: Could not find the scale factor for the stimset None, using 1.0 as fallback.
  warnings.warn(
/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:550: UserWarning: Could not find settings for amplifier unknown of channel Vm_primary.
  warnings.warn(f"Could not find settings for amplifier {amplifier} of channel {adcName}.")
/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:605: UserWarning: Unsupported clamp mode {clampMode}
  warnings.warn("Unsupported clamp mode {clampMode}")
/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:1

NWB Conversion complete for data/processed/recordings/human2019/19o10065.nwb
Subject meta: {'Subject': {'age': None, 'description': "['C1, L2-RMP -64']", 'species': 'Homo sapiens', 'subject_id': 'Donor_2019_09_03_0017'}}
Stimulus chan: I_clampsec
Response chan: Vm_primary
****************************************************************************************************
Error: 'bias_current' 
 data/raw/recordings/Human tissue/Human tissue-White noise_2019/Sep_03-2019/C1_L2/Intrinsic Properties_C1_L2/2019_09_03_0017.abf 

Subject meta: {'Subject': {'age': None, 'description': "['C1, L2-RMP -64']", 'species': 'Homo sapiens', 'subject_id': 'Donor_2019_09_03_0020'}}
Stimulus chan: Cmd 1
Response chan: Vm_sec
NWB Conversion complete for data/processed/recordings/human2019/2019_09_03_0020.nwb


In [35]:
!dandi validate "data/processed/recordings/human2019/"

2021-07-28 15:39:27,015 [ WARNING] A newer version (0.25.0) of dandi/dandi-cli is available. You are using 0.23.2
2021-07-28 15:39:30,371 [    INFO] data/processed/recordings/human2019/19320017.nwb: ok
2021-07-28 15:39:32,146 [    INFO] data/processed/recordings/human2019/19320001.nwb: ok
2021-07-28 15:39:33,846 [    INFO] data/processed/recordings/human2019/2019_11_28_0010.nwb: ok
2021-07-28 15:39:35,485 [    INFO] data/processed/recordings/human2019/2019_11_28_0038.nwb: ok
2021-07-28 15:39:37,187 [    INFO] data/processed/recordings/human2019/2019_11_26_0019.nwb: ok
2021-07-28 15:39:38,954 [    INFO] data/processed/recordings/human2019/19228058.nwb: ok
2021-07-28 15:39:40,763 [    INFO] data/processed/recordings/human2019/19129072.nwb: ok
2021-07-28 15:39:42,582 [    INFO] data/processed/recordings/human2019/19129058.nwb: ok
2021-07-28 15:39:44,463 [    INFO] data/processed/recordings/human2019/19320007.nwb: ok
2021-07-28 15:39:46,257 [    INFO] data/processed/recordings/human2019/20

## Extract stimulus info to later merge with metadat

In [44]:
converted_nwb_paths = list(Path('data/processed/recordings/human2019/').glob('*.nwb'))

In [45]:
converted_nwb_file_ids = [x.stem for x in converted_nwb_paths]

In [46]:
to_extract_stim_info = human2019_file_info[human2019_file_info.file_id.isin(converted_nwb_file_ids)]

In [47]:
# parse relevant info related to stimulus, including duration, and amplitudes
def get_stim_info(abf_path, stim_channel_num = 1, stim_gain = 1, stim_name = 'sweepC'):
    abf = pyabf.ABF(abf_path)
    num_sweeps = abf.sweepCount
    stim_amps = np.zeros(num_sweeps) 
    stim_start_time = None
    stim_end_time = None
    sampling_rate = int(round(1/(abf.sweepX[2] - abf.sweepX[1]))) # manually calculate the sampling rate

    for i in range(0, num_sweeps):
        abf.setSweep(i, channel=stim_channel_num)
        sampling_rate = abf.dataRate
        if stim_name == 'sweepY':
            stim_vec = np.round(abf.sweepY * stim_gain)
        else:
            stim_vec = np.round(abf.sweepC * stim_gain)
        stim_amp = stim_vec[5000]

        stim_amps[i] = round(stim_amp)
        non_zero_inds = np.where(stim_vec == stim_amp)
        stim_duration = np.shape(non_zero_inds)[1] * 1/sampling_rate
        if stim_duration == 0:
            continue
        stim_start_ind = non_zero_inds[0][0]
        stim_end_ind = non_zero_inds[0][-1]
        
        stim_start_time = abf.sweepX[stim_start_ind]
        stim_end_time = abf.sweepX[stim_end_ind]

    ret_dict = {'stim_amp_vec' : stim_amps, 'stim_duration' : stim_duration, 
                'stim_start_time' : stim_start_time, 'stim_end_time' : stim_end_time, 'num_sweeps' : num_sweeps,
               'stim_sampling_rate' : sampling_rate}
    return(ret_dict)
    """
    return pd.Series([stim_amps, stim_duration, stim_start_time, stim_end_time, num_sweeps, sampling_rate], 
                     index=['stim_amp_vec', 'stim_duration', 'stim_start_time', 'stim_end_time', 'num_sweeps', 'stim_sampling_rate'])
    """

In [48]:
stim_info = {}
for i, row in to_extract_stim_info.iterrows():
    try:
        abf_info = get_stim_info(row.path, stim_name = 'sweepC')
    except (AttributeError, ValueError) as e:
        print(e)
        abf_info = None
    stim_info[row.file_name] = abf_info

In [49]:
stim_info_df = pd.DataFrame(stim_info).T

In [50]:
stim_info_dir = Path('./data/processed/meta/stiminfo/')
stim_info_dir.mkdir(exist_ok=True)

In [51]:
stim_info_df.index.name = 'abf_filename'
stim_info_df.to_csv(stim_info_dir / 'human2019_stiminfo.csv')